In [10]:
import codecs
import pandas as pd

In [11]:
nomLexique3="/Users/gilles/Dropbox/Travail/Recherche/BoKu/LingInfo/Lexique3/Lexique381/Lexique381.csv"

In [12]:
dfLex3Brut=pd.read_csv(nomLexique3,sep="\t")

In [25]:
lex3Categories=[c for c in dfLex3Brut["4_cgram"].unique().tolist() if isinstance(c,str)]
print ", ".join(sorted(lex3Categories))

ADJ, ADJ:dem, ADJ:ind, ADJ:int, ADJ:num, ADJ:pos, ADV, ART:def, ART:ind, AUX, CON, LIA, NOM, ONO, PRE, PRO:dem, PRO:ind, PRO:int, PRO:per, PRO:pos, PRO:rel, VER


In [49]:
dfNoms=dfLex3Brut[dfLex3Brut["4_cgram"]=="NOM"].loc[:,"1_ortho":"6_nombre"]
dfNoms.count()["1_ortho"]

48286

In [50]:
dfAdjectifs=dfLex3Brut[dfLex3Brut["4_cgram"]=="ADJ"].loc[:,"1_ortho":"6_nombre"]
dfAdjectifs.count()["1_ortho"]

26806

In [56]:
dfNoms.groupby("5_genre").count()
dfNoms.groupby("6_nombre").count()
dfNoms.groupby(["5_genre","6_nombre"]).count()[["3_lemme"]]

3_lemme
5_genre 6_nombre         
f       p            7069
        s           11787
m       p            9673
        s           15351

In [54]:
dfNoms[dfNoms["5_genre"].isnull()]

,1_ortho,2_phon,3_lemme,4_cgram,5_genre,6_nombre
271,abolitionniste,abolisjonist,abolitionniste,NOM,NaN,s
273,abolitionnistes,abolisjonist,abolitionniste,NOM,NaN,p
357,aborigène,aboRiZEn,aborigène,NOM,NaN,s
359,aborigènes,aboRiZEn,aborigène,NOM,NaN,p
378,aboulique,abulik,aboulique,NOM,NaN,s
598,absentéiste,aps@teist,absentéiste,NOM,NaN,s
676,abstentionniste,apst@sjOnist,abstentionniste,NOM,NaN,s
677,abstentionnistes,apst@sjonist,abstentionniste,NOM,NaN,p
727,absurdistes,apsyRdist,absurdiste,NOM,NaN,p
975,accessoiriste,akseswaRist,accessoiriste,NOM,NaN,s


In [38]:
dfNoms[dfNoms["5_genre"]=="m"]

,1_ortho,2_phon,3_lemme,4_cgram,5_genre,6_nombre,7_freqlemfilms2,8_freqlemlivres,9_freqfilms2,10_freqlivres,...,26_orthrenv,27_phonrenv,28_orthosyll,29_cgramortho,30_deflem,31_defobs,32_old20,33_pld20,34_morphoder,35_nbmorph
0,a,a,a,NOM,m,NaN,81.36,58.65,81.36,58.65,...,a,a,a,"NOM,AUX,VER",NaN,NaN,1.0,1.00,a,1
12,a priori,apRijoRi,a priori,NOM,m,NaN,0.41,0.47,0.41,0.47,...,iroirp a,iRojiRpa,a prio-ri,"ADV,NOM",93.0,16.0,3.8,3.25,a-priori,2
13,aa,aa,aa,NOM,m,s,0.01,0.00,0.01,0.00,...,aa,aa,NaN,NOM,NaN,NaN,1.0,1.00,aa,1
17,abaca,abaka,abaca,NOM,m,s,0.01,0.00,0.01,0.00,...,acaba,akaba,NaN,NOM,NaN,NaN,2.0,1.90,abaca,1
27,abaisse-langue,abEsl@g,abaisse-langue,NOM,m,NaN,0.04,0.14,0.04,0.14,...,eugnal-essiaba,g@lsEba,abaisse-langue,NOM,NaN,NaN,6.3,2.85,abaisse-langue,2


# Combinaisons de traits

In [71]:
def combinaisonsTraits(dfLocal):
    genres={}
    result={}
    for g in "mf":
        genres[g]=dfLocal[dfLocal["5_genre"]==g]
    genres["nan"]=dfLocal[dfLocal["5_genre"].isnull()]
    for g in genres:
        for n in "sp":
            result[(g,n)]=genres[g][genres[g]["6_nombre"]==n]
        result[(g,"nan")]=genres[g][genres[g]["6_nombre"].isnull()]
    return result

In [123]:
def calculerCouvertureNoms(dfLocal):
    nbFormes=0
    for k,v in combinaisonsTraits(dfLocal).iteritems():
        print k,v.count()["3_lemme"]
        if not "nan" in k:
            nbFormes+=v.count()["3_lemme"]
        else:
            if k==("nan","nan"):
                nbFormes+=4*v.count()["3_lemme"]
            else:
                nbFormes+=2*v.count()["3_lemme"]
    print
    print "nb formes",nbFormes
#    len(pd.unique(dfNoms[["3_lemme","5_genre"]].values.ravel("K")))

    dfEpicenes=dfLocal[dfLocal["5_genre"].isnull()]
    dfGenres=dfLocal[~dfLocal["5_genre"].isnull()]

    nbEpicenes=len(pd.unique(dfEpicenes[["3_lemme","5_genre"]].values.ravel("K")))
    nbGenres=len(pd.unique(dfGenres[["3_lemme","5_genre"]].values.ravel("K")))
    nbTotal=nbEpicenes*2+nbGenres

    print u"épicènes",nbEpicenes
    print u"genrés",nbGenres
    print u"nb total",nbTotal, "nb cases",nbTotal*2
    print u"couverture %2.1f"%(100*float(nbFormes)/(nbTotal*2))

In [127]:
def calculerCouvertureAdjectifs(dfLocal):
    nbFormes=0
    for k,v in combinaisonsTraits(dfLocal).iteritems():
        print k,v.count()["3_lemme"]
        if not "nan" in k:
            nbFormes+=v.count()["3_lemme"]
        else:
            if k==("nan","nan"):
                nbFormes+=4*v.count()["3_lemme"]
            else:
                nbFormes+=2*v.count()["3_lemme"]
    print
    print "nb formes",nbFormes
    nbTotal=len(pd.unique(dfLocal[["3_lemme"]].values.ravel("K")))
    print u"nb total",nbTotal, "nb cases",nbTotal*4
    print u"couverture %2.1f"%(100*float(nbFormes)/(nbTotal*4))

In [128]:
calculerCouvertureAdjectifs(dfAdjectifs)

('m', 'nan') 537
('m', 's') 6439
('f', 'nan') 4
('nan', 'nan') 288
('f', 's') 5981
('nan', 's') 2639
('m', 'p') 4777
('f', 'p') 4423
('nan', 'p') 1718

nb formes 32568
nb total 11580 nb cases 46320
couverture 70.3
